In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

ModuleNotFoundError: No module named 'torch'

In [2]:
class Model(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_layers, batch_size, seq_len, batch_first = True):
        super(Model, self).__init__()
        self.LSTM = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first)
        self.inp = torch.randn(batch_size, seq_len, input_dim)
        
        #We'll have to initialize a hidden state and cell state for the LSTM as this is the first cell.
        #The hidden state and cell state is stored in a tuple with the format (hidden_state, cell_state).
        self.hidden_state = torch.randn(n_layers, batch_size, hidden_dim)
        self.cell_state = torch.randn(n_layers, batch_size, hidden_dim)
        self.hidden = (self.hidden_state, self.cell_state)
        
    def many_to_one(self):
        out, hidden = self.LSTM(self.inp, self.hidden)
        return out, hidden
        
    
    def many_to_many(self, out):
        out = out.squeeze()[-1, :]
        return out
        

NameError: name 'nn' is not defined

In [37]:
input_dim = 5 #Input dimension: represents the size of the input at each time step. 
hidden_dim = 10 #Hidden dimension: represents the size of the hidden state and cell state at each time step.
n_layers = 1 #The number of LSTM layers stacked on top of eachother.

In [43]:
#dummy data

#our input dimension is 5, we have to create a tensor of the shape (1, 1, 5)
#this represents (batch size, sequence length, input dimension)

batch_size = 1
seq_len = 3


In [44]:
lstm = Model(input_dim, hidden_dim, n_layers, batch_size, seq_len)
out, hidden = lstm.many_to_one()
print("output shape: ", out.shape)
print("Hidden: ", hidden)
print(lstm.many_to_many(out))

RuntimeError: Expected hidden[0] size (1, 3, 10), got (1, 1, 10)